### Generate an environment
Currently, this just copies data/default_env.pickle into data/env.pickle.

**To do**: Add a DVC parameter such as `env_name` and if this isn't set to "default" then generate a new environment.

In [ ]:
!dvc run --force -n gen_env -w .. -d data/default_env.pickle -o data/env.pickle python scripts/gen_env.py

### Generate observations
Currently this checks the values of the parameters `target_area`, `true_norm`, `num_observations` and `obs_data_set`. If they are equal to the settings of the observations in data/default_observations.pickle (and `obs_data_set` has value 1), then that file is copied into data/observations.pickle.

**To do**: Generate a new set of observations if the parameter settings don't match the default one, or `obs_data_set` is not 1.


In [ ]:
!dvc run --force -n gen_observations -w .. -d data/env.pickle -p target_area,true_norm,num_observations,obs_data_set -o data/observations.pickle python scripts/gen_observations.py

### Generate MCMC chains
Creates data/chains_and_log_likelihoods.pickle. Also metrics/starts_info.txt, but that is not currently recorded in the "dvc run" command below.

In [4]:
!dvc run --force --always-changed -n gen_mcmc_chains -w .. -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o data/chains_and_log_likelihoods.pickle python scripts/gen_mcmc_chains.py

E_new rejected
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54381.30828906423
log(factor) = 0
log(p_accept)=0.0000000
E_new accepted
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54491.169517931026
log(factor) = 0
log(p_accept)=-109.8612289
E_new rejected
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54710.891975664614
log(factor) = 0
log(p_accept)=-329.5836866
E_new rejected
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54930.614433398194
log(factor) = -0.5108256237659907
log(p_accept)=-549.8169700
E_new rejected
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54381.30828906423
log(factor) = 0
log(p_accept)=0.0000000
E_new accepted
Old Expression: Log-Likelihood=-54381.30828906423
New Expression: Log-Likelihood=-54930.614433398194
log(factor) = -0.5108256237659907
log(p_accept)=-549.8169700
E_new rejected
Old

### Analyse the chains
Generate file metrics/chain_likelihoods.csv and append more information to metrics/chain_info.txt (note: DVC may not handle changing a file very well - perhaps a separate file should be created).

**Temporary hack**: chain_likelihoods contains posterior probabilities now, even though the field name has not changed.

In [5]:
!dvc run --force --always-changed -n analyse_chains -w .. -d data/chains_and_log_likelihoods.pickle -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o metrics/chain_info.txt --plots metrics/chain_likelihoods.csv python scripts/analyse_chains.py


('Norms', ('Obl', ('Moved', ('Colour', 'b'), ('Shape', 'square'), ('Zone', '2'), ('Next-Move', ('Colour', 'b'), ('Shape', 'triangle'))), ('Zone', '2')), ('Per', ('Action', 'putdown'), ('Colour', 'b'), ('Shape', 'square'), ('PerZone', '2'))) in chains list: [False, False, False, True, False]

('Norms', ('Obl', ('Moved', ('Colour', 'any'), ('Shape', 'any'), ('Zone', '3'), ('Moved', ('Colour', 'b'), ('Shape', 'any'), ('Zone', '1'), ('Next-Move', ('Colour', 'b'), ('Shape', 'any')))), ('Zone', '2')), ('Per', ('Action', 'putdown'), ('Colour', 'r'), ('Shape', 'circle'), ('PerZone', '1'))) in chains list: [False, False, False, False, True]

('Norms', ('Obl', ('Moved', ('Colour', 'b'), ('Shape', 'square'), ('Zone', '1'), ('Moved', ('Colour', 'r'), ('Shape', 'circle'), ('Zone', '2'), ('Next-Move', ('Colour', 'r'), ('Shape', 'circle')))), ('Zone', '2')), ('Per', ('Action', 'putdown'), ('Colour', 'g'), ('Shape', 'any'), ('PerZone', '2'))) in chains list: [True, False, False, False, False]

('Norm

### Generate a plot showing norms ordered by likelihood (or now posterior prob.?)
The file plots.html is generated in the root folder, based on data in metrics/chain_likelihoods.csv

In [7]:
!pushd ..; dvc plots show -t plots/norm_exp_histogram.json metrics/chain_likelihoods.csv

~/work ~/work/notebooks
file:///home/jovyan/work/plots.html


### Perform a convergence test to compute $\hat{R}$
The result is written to metrics/conv_test.txt. Also, the posterior sample (after discarding initial warm-up segments of chains and combining the remains) is writen to data/posterior.pickle. 


In [36]:
!dvc run --force --always-changed -n conv_test -w .. -d data/chains_and_log_likelihoods.pickle -p rhat_step_size -o data/posterior.pickle -o metrics/conv_test.txt python scripts/conv_test.py

Running stage 'conv_test' with command:                               core>
	python scripts/conv_test.py

Calculating Variance for 1st 50 iterations
Temp. check: seq length is 50
Time taken for job=0.9s

Calculating Variance for 1st 100 iterations
Temp. check: seq length is 100
Time taken for job=1.4s

Calculating Variance for 1st 200 iterations
Temp. check: seq length is 200
Time taken for job=3.0s

Calculating Variance for 1st 400 iterations
Temp. check: seq length is 400
Time taken for job=6.2s

Calculating Variance for 1st 499 iterations
Temp. check: seq length is 499
Time taken for job=8.0s

Modifying stage 'conv_test' in 'dvc.yaml'                             core>
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


### Extract the top norms and compute precision and recall
The results are written to metrics/precision_recall.txt.


In [9]:
!dvc run --force --always-changed -n extract_top_norms -w .. -d data/posterior.pickle -d data/env.pickle -p colour_specific,shape_specific,target_area,true_norm -o metrics/precision_recall.txt python scripts/extract_top_norms.pyComputing Machine Laboratory

Running stage 'extract_top_norms' with command:                       core>
	python scripts/extract_top_norms.py

Calculting P&R for exp at rank 1

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 2

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 3

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 4

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 5

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 6

calculate_pr: len(true)=10000; len(predicted)=10000
Identical lists: False
len(intersection)=10000
Calculting P&R for exp at rank 7

calculate_pr: len(true)=10000; len(predicted)=10000
Id

### The cells below are for testing and are not part of the workflow

In [ ]:
!cd ..; python scripts/test_overdispersed_starts.py

Time to initialise E_0=11.9123s
Time to initialise E_0=13.7513s
Time to initialise E_0=25.6745s
Time to initialise E_0=58.1967s
Time to initialise E_0=8.7295s
Time to initialise E_0=91.3707s
Time to initialise E_0=38.2951s
Time to initialise E_0=187.9690s
Time to initialise E_0=29.0857s
Time to initialise E_0=133.3056s
Time to initialise E_0=4.4487s
Time to initialise E_0=229.6389s
Time to initialise E_0=27.4963s
Time to initialise E_0=8.9403s
Time to initialise E_0=162.3387s
Time to initialise E_0=87.5131s
Time to initialise E_0=74.2847s
Time to initialise E_0=34.4173s
Time to initialise E_0=199.9201s
Time to initialise E_0=90.5431s
Time to initialise E_0=272.6284s
Time to initialise E_0=39.4856s
Time to initialise E_0=64.4547s
Time to initialise E_0=296.4189s
Time to initialise E_0=100.0625s
Time to initialise E_0=38.8014s
Time to initialise E_0=56.6444s
Time to initialise E_0=177.6544s
Time to initialise E_0=296.9581s
Time to initialise E_0=40.4466s
Time to initialise E_0=57.1393s
T

In [ ]:
!dvc repro

In [ ]:
!rm ../.dvc/tmp/rwlock

In [39]:
!cd ..; python scripts/conv_test_no_warmup.py


Calculating Variance for 1st 50 iterations
Temp. check: seq length is 50
Time taken for job=0.8s

Calculating Variance for 1st 100 iterations
Temp. check: seq length is 100
Time taken for job=1.5s

Calculating Variance for 1st 200 iterations
Temp. check: seq length is 200
Time taken for job=3.2s

Calculating Variance for 1st 400 iterations
Temp. check: seq length is 400
Time taken for job=6.5s

Calculating Variance for 1st 800 iterations
Temp. check: seq length is 800
Time taken for job=14.1s

Calculating Variance for 1st 1000 iterations
Temp. check: seq length is 1000
Time taken for job=18.0s

   iterations         R  within_seq_var  between_seq_var  var_over_est
0          50  1.037624        5.541469       430.372000      5.966300
1         100  1.057835        6.361455       763.463778      7.118557
2         200  1.093666        7.372869      1453.228111      8.818725
3         400  1.159839        7.997286      2768.877556     10.758166
4         800  1.301818        7.882874   

In [35]:
import os
os.getcwd()